In [1]:
import random
import pyspark
from pyspark.sql import SparkSession, functions
from pyspark.sql.functions import concat

import ConnectionConfig as cc
from pyspark.sql.functions import *
cc.setupEnvironment()
cc.listEnvironment()
cc.set_connectionProfile("veloDB")


ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 2468
JPY_INTERRUPT_EVENT: 2468
JPY_PARENT_PID: 2480
JPY_SESSION_NAME: S1_02a_DIM_USER_INITIAL.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-

In [2]:
spark = cc.startLocalCluster("dimUsersInit")
spark.getActiveSession()

In [3]:
#EXTRACT
df_operational_users_dim = spark.read \
    .format("jdbc") \
    .option("driver", cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 1) \
    .option("upperBound", 60000) \
    .load()

In [4]:
#TRANSFORM
df_operational_users_update = (df_operational_users_dim
                               .withColumn("userid", col("userid"))
                               .withColumn("user_sk", expr("uuid()"))
                               .withColumn("scd_start", lit("1990-01-01").cast("timestamp"))
                               .withColumn("scd_end", lit("2100-12-12").cast("timestamp"))
                               .withColumn("md5", md5(concat('street', 'number', 'zipcode', 'city')))
                               .withColumn("current", lit(True)))

df_operational_users_update.printSchema()
df_operational_users_update.show(5)


root
 |-- userid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- user_sk: string (nullable = false)
 |-- scd_start: timestamp (nullable = true)
 |-- scd_end: timestamp (nullable = true)
 |-- md5: string (nullable = true)
 |-- current: boolean (nullable = false)

+------+-----------------+--------------------+--------------------+--------+-------+-------------------+------------+--------------------+-------------------+-------------------+--------------------+-------+
|userid|             name|               email|              street|  number|zipcode|               city|country_code|             user_sk|          scd_start|            scd_end|                 md5|current|
+------+-----------------+--------------------+--------------------

In [5]:
# Save as a Delta table (first time creation)
#LOAD
df_operational_users_update.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("user_dim_current")


In [6]:
# Describe the Delta table and find the location
spark.sql("DESCRIBE FORMATTED user_dim_current").show(truncate=False)


+----------------------------+-----------------------------------------------------------------+-------+
|col_name                    |data_type                                                        |comment|
+----------------------------+-----------------------------------------------------------------+-------+
|userid                      |int                                                              |NULL   |
|name                        |string                                                           |NULL   |
|email                       |string                                                           |NULL   |
|street                      |string                                                           |NULL   |
|number                      |string                                                           |NULL   |
|zipcode                     |string                                                           |NULL   |
|city                        |string                   